In [54]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')
papers_info = pd.read_csv('citations.csv')
papers_info['id'] = range(len(papers_info))
papers_info['name'] = papers_info['Title']

paper = open('paper_citations.txt','r')
paper_citation = paper.readlines()
paper.close()
#用bs4解析html
from bs4 import BeautifulSoup
paper_citation = BeautifulSoup(paper_citation[0], 'html.parser')
#找到所有的引用
paper_citation = paper_citation.find_all('tr', class_='gsc_a_tr')
papers = []
for i in range(len(paper_citation)):
    Title = paper_citation[i].find('a', class_='gsc_a_at').text
    Authors = paper_citation[i].find('div', class_='gs_gray').text
    Journal = paper_citation[i].find('div', class_='gs_gray').find_next('div', class_='gs_gray').text
    Cite = paper_citation[i].find('a', class_='gsc_a_ac').text
    if Cite == '':
        Cite = 0
    papers.append({
        'name':Title,
        'Cite':int(Cite)})
papers = pd.DataFrame(papers)
papers = pd.merge(papers_info, papers, on='name', how='left')

In [81]:
def getlink(df):
    df1 = df[['name','Year']].rename(columns={'name':'source','Year':'sourceYear'})
    df1['flag'] = 1
    df2 = df1.copy().rename(columns={'source':'target','sourceYear':'targetYear'})
    df = pd.merge(df1,df2,on='flag').drop('flag',axis=1) 
    df = df[(df['targetYear']>df['sourceYear'])&(df['targetYear']<=df['sourceYear']+1)]
    return df

In [ ]:
papers['Authors'] = papers['Authors'].apply(lambda x: x.split(';'))


In [114]:
papers_sub = papers.explode('Authors')
papers_sub = papers_sub[~papers_sub['Authors'].isin(['Yu, Qing','余庆'])]
paper_links = pd.merge(papers_sub,papers_sub,on='Authors').groupby(['Title_x','Title_y','category_x','category_y','Year_x','Year_y']).size().reset_index()
paper_links = paper_links[(paper_links['Title_x']!=paper_links['Title_y'])&(paper_links[0]>1)]
paper_links = paper_links[(paper_links['Year_y']>=paper_links['Year_x'])]
# 一年内同类型，共同作者大于2
condition1 = (paper_links['Year_y']<paper_links['Year_x']+1)&(paper_links['category_x']==paper_links['category_y'])&(paper_links[0]>=2)
# 两年内同类型，共同作者大于3
condition2 = (paper_links['Year_y']<paper_links['Year_x']+2)&(paper_links['category_x']==paper_links['category_y'])&(paper_links[0]>=3)
# 三年内不同类型，但是共同作者大于4
condition3 = (paper_links['Year_y']<paper_links['Year_x']+3)&(paper_links['category_x']!=paper_links['category_y'])&(paper_links[0]>=4)
paper_links = paper_links[condition1|condition2|condition3]
paper_links.rename(columns={'Title_x':'source','Title_y':'target',0:'weight'}).to_json('paperlinks.json',orient='records')

In [82]:
papers.sort_values(['category','Year']).groupby('category').apply(getlink)[['source','target']].to_json('paperlinks.json',orient='records')

In [53]:
papers.to_json('papers.json', orient='records',force_ascii=False)
papers

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,category,id,name,Cite
0,"Yu, Qing; Li, Weifeng; Duan, Zhengyu; Yang, Do...",Unveiling Urban Commuting Structure from Mobil...,NaN,NaN,NaN,NaN,2018.0,NaN,Human Mobility,0,Unveiling Urban Commuting Structure from Mobil...,1
1,"Yu, Qing; Li, Wei-Feng; Duan, Zheng-Yu; Yang, ...",Activity Space Investigations for Nonresidents...,CICTP 2017: Transportation Reform and Change—E...,NaN,NaN,142-151,2018.0,"American Society of Civil Engineers Reston, VA",Human Mobility,1,Activity Space Investigations for Nonresidents...,0
2,余庆; 雷曾翔; 李玮峰; 杨东援;,基于多源数据的公交客流规模关联分析,综合运输,39.0,6.0,58-63,2017.0,NaN,Public Transport,2,基于多源数据的公交客流规模关联分析,0
3,李俊; 杨东援; 余庆;,基于 POI 数据的配送企业网络布局,2017 年中国城市交通规划年会论文集,NaN,NaN,NaN,2017.0,NaN,Logistics,3,基于 POI 数据的配送企业网络布局,0
4,李玮峰; 程小云; 余庆; 杨东援;,基于多源数据的居民活动区域特征分析,规划 60 年: 成就与挑战——2016 中国城市规划年会论文集 (04 城市规划新技术应用),NaN,NaN,NaN,2016.0,NaN,Human Mobility,4,基于多源数据的居民活动区域特征分析,0
...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Jiao, Qianhui; Wang, Jinghui; Cheng, Long; Che...",Carbon emission reduction effects of heterogen...,Applied Energy,379.0,NaN,124826,2025.0,Elsevier,Transportation Policy,62,Carbon emission reduction effects of heterogen...,0
63,"Yu, Qing; Li, Jiaxing; Feng, Defan; Liu, Xuany...",Modeling electric vehicle behavior: Insights f...,NaN,NaN,NaN,NaN,2025.0,"Applied Energy 380, 125066",Electric Vehicle,63,Modeling electric vehicle behavior: Insights f...,1
64,"Yu, Qing; Dong, Kechuan; Guo, Zhiling; Li, Jia...",Global Estimation of Building-Integrated Facad...,arXiv preprint arXiv:2412.01291,NaN,NaN,NaN,2024.0,NaN,Renewable energy,64,Global Estimation of Building-Integrated Facad...,0
65,"Liu, Xuanyu; Yu, Qing; Bian, Weihao; Yu, Hao; ...",Unraveling spatiotemporal dynamics of rideshar...,Transportation Research Part D: Transport and ...,139.0,NaN,104594,2025.0,Elsevier,Ride Sharing,65,Unraveling spatiotemporal dynamics of rideshar...,0
